In [27]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

Path = '/content/gdrive/My Drive/Colab/Nails/data'
PathTrain = join(Path, 'train_one_class')

def adjustData(img,mask):
    if(np.max(img) > 1):
        img = img / 255
        mask = mask / 255
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
    return (img,mask)

def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode = "grayscale",
                    mask_color_mode = "grayscale", image_save_prefix  = "image", mask_save_prefix  = "mask",
                    save_to_dir = None, target_size = (256,256), seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        seed = seed)
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        seed = seed)
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask)
        yield (img,mask)

def testData(test_path, target_size = (256,256), flags = cv2.IMREAD_GRAYSCALE):
  onlyfiles = [f for f in listdir(test_path) if isfile(join(test_path, f))]
  result = []
  file_names = []
  for f in onlyfiles:
    file_names.append(f)
    img = cv2.imread(join(test_path, f), flags)
    img = img / 255
    img = cv2.resize(img, target_size, interpolation = cv2.INTER_AREA)
    img = np.reshape(img, (1,) + img.shape + (1,))
    result.append(img)
  return result, file_names

def saveResult(npMask, save_path, fileName, original_path, train_path = ''):
  if not os.path.exists(join(PathTrain, save_path)):
    os.makedirs(join(PathTrain, save_path))
  img = npMask[0,:,:,0]
  img_original = cv2.imread(join(original_path, fileName))
  img = cv2.resize(img, (img_original.shape[1], img_original.shape[0]))    
  img = (img*(256/img.max())).astype(int)  
  img_original[:,:,0] = (img_original[:,:,0]).astype(int)
  img_original[:,:,1] = (img_original[:,:,1] / (1 + (img / 192))).astype(int)
  img_original[:,:,2] = (img_original[:,:,2] / (1 + (img / 192))).astype(int)
  if train_path != '':
    train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
    if fileName in train_files:
      cv2.putText(img_original,'train',(20,20),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2)
  cv2.imwrite(join(join(PathTrain, save_path),fileName),img_original)

In [0]:

from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
def train(checkpoint_to_save, steps_per_epoch, epochs, checkpoint_to_load = ''):
  data_gen_args = dict(rotation_range=0.6,
                      width_shift_range=0.1,
                      height_shift_range=0.1,
                      shear_range=0.1,
                      zoom_range=0.1,
                      horizontal_flip=True,
                      fill_mode='nearest')
  
  #image_color_mode: grayscale, rgb
  myGene = trainGenerator(1,PathTrain,'pics','masks',data_gen_args,save_to_dir = None,image_color_mode = 'grayscale')
  model = unet()
  if checkpoint_to_load != '':
    model.load_weights(join(PathTrain, checkpoint_to_save))
  model_checkpoint = ModelCheckpoint(join(PathTrain, checkpoint_to_save), monitor='loss',verbose=1, save_best_only=True)
  model.fit_generator(myGene,steps_per_epoch,epochs,callbacks=[model_checkpoint])

In [0]:
def test(checkpoint, results_path):
  #flags: IMREAD_COLOR, IMREAD_GRAYSCALE
  X, file_names = testData(Path + '/test', flags = cv2.IMREAD_GRAYSCALE )
  model = unet()
  model.load_weights(join(PathTrain, checkpoint))
  for fileName, x in zip(file_names, X):
      result = model.predict(x, verbose=0)
      saveResult(result, results_path, fileName, Path + '/test', PathTrain + '/pics')

In [32]:
#train('unet_membrane.hdf5', 100, 1)
test('unet_membrane.hdf5', 'test_results_one_class')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
for i in range(100):
  print('step ' + str(i))
  train('unet_membrane.hdf5', 1000, 1, 'unet_membrane.hdf5')
  test('unet_membrane.hdf5', 'test_results_one_class')

step 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Epoch 1/1
Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
  85/1000 [=>............................] - ETA: 5:08 - loss: 6.0252e-04 - acc: 0.9998